# FIE for Dirichlet BVP in a circle with Polynomial Gauss rule

In [ ]:
import numpy as np
import scipy.integrate as integrate
#import scipy.interpolate as interp
%matplotlib inline
import matplotlib.pyplot as plt
import scipy
import math
import time


In [ ]:
def K_matrix(phis, taus):
    """Returns the Kernel for BIE as bivariate function
    """
    return np.ones((len(phis), len(taus))) * 1/(2*np.pi)


def ke(r, phi, theta):
    """Returns the kernel function to find 
    the solution of BIE for the hole space -
    three-variate function"""
    kernel = (1 - r*np.cos(theta-phi)) / \
                 (r**2 + 1 - 2*r*np.cos(theta-phi))
    return kernel


def h(phi):
    """Returns the right-hand side univariate function 
    in BIE"""
    return np.cos(2*phi)


def E(r, phi):
    """The exact solution of the Dirichlet BVP
    for Laplace equation"""
    return (r**2) * np.cos(2*phi)

In [ ]:
def taus_omegas_Spline54(a, b, x, w):
    
    taus_n = (b - a)/10 * x + a
    omegas_n = w*(b - a)/10
    
    omegas = np.append(omegas_n[:-1], omegas_n[::-1])
    taus = taus_n - taus_n.max() 
    taus = np.append(taus_n[:-1], -taus[::-1] + taus_n[-1])
    
    return taus, omegas 

## Spline 54

In [ ]:
def test(N):
    a = 0
    b = 2 * np.pi

    print(N)    # number of macroblocks
    order = 21   # QP initial

    phis = np.linspace(0, 2*np.pi, 300)
    rs =   np.linspace(0, 1, 100, endpoint=False)
    
    #### define taus, omegas for integration with macroblocks

    with open('TargetNodes_d=5_k=4_N=10.txt', 'r') as ff:
        x = np.loadtxt(ff)
    with open('TargetWeights_d=5_k=4_N=10.txt', 'r') as ff:
        w = np.loadtxt(ff)
    
    interval = np.linspace(a, b, N+1)
    taus = []; omegas = []
    for a_, b_ in zip(interval[:-1], interval[1:]):
        t_, o_ = taus_omegas_Spline54(a_, b_, x, w)
        taus.append(t_)
        omegas.append(o_)
    taus = np.ravel(taus)
    omegas = np.ravel(omegas)
    #### block matrix and solution for sigmas

    Ko_blocks = []
    for taus_, omegas_ in zip(taus.reshape((N, order)),
                              omegas.reshape((N, order))):
        Ko_blocks.append(K_matrix(taus_, taus_) * omegas_)
    Ko = scipy.linalg.block_diag(*Ko_blocks)

    sigmas = np.dot(np.linalg.inv(np.identity(len(taus)) + Ko), 2*h(taus))

    #### Nystrom algorithm

    sigmas_ns = 2 * h(taus) - (K_matrix(taus, taus) * omegas) @ sigmas
    
    
    #### final integration

    rs_m, phis_m, thetas_m = np.meshgrid(rs, phis, taus, indexing='ij')
    ke_matrix = ke(rs_m, phis_m, thetas_m)

    integral = ((ke_matrix * sigmas_ns) * omegas).sum(axis=-1)

    rs_m, phis_m = np.meshgrid(rs, phis, indexing='ij')
    x_mesh, y_mesh = rs_m * np.cos(phis_m), rs_m * np.sin(phis_m)
    u_mesh = 1/(2*np.pi) * integral

    E_mesh = E(rs_m, phis_m)
    
    # l2errors = np.sqrt(np.sum(np.power((E_mesh - u_mesh)*(phis[1]-phis[0])*(rs[1]-rs[0]),2)))

    return (abs(E_mesh - u_mesh)).max(), E_mesh - u_mesh, E_mesh, x_mesh, y_mesh

In [ ]:
errors,error_mesh, E_mesh, x_mesh, y_mesh = test(16)

In [ ]:
import pickle
with open('spline5_336_cos2phi.pkl', 'wb')as ff:
    pickle.dump(error_mesh, ff)